In [1]:
!pip install --upgrade xgboost optuna scikit-learn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
import pandas as pd
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from math import sqrt

from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv("final_train.csv")

In [4]:
df

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Episode_Title_num,Ad_Density,Popularity_Diff,Popularity_Interaction,Host_Popularity_squared,Popularity_Average
0,0,34,98,63.84,9,74.81,4,3,3.0,0.0,1,31.41998,98,0.000000,71.81,224.43,5596.5361,38.905
1,1,24,19,119.80,1,66.95,2,0,3.0,2.0,-1,88.01241,26,0.016694,63.95,200.85,4482.3025,34.975
2,2,40,8,73.90,2,69.97,5,1,3.0,0.0,-1,44.92531,16,0.000000,66.97,209.91,4895.8009,36.485
3,3,10,40,67.17,8,57.22,1,2,3.0,2.0,1,46.27824,45,0.029775,54.22,171.66,3274.1284,30.110
4,4,31,85,110.51,3,80.07,1,0,3.0,3.0,0,75.61031,86,0.027147,77.07,240.21,6411.2049,41.535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,26,18,75.66,2,69.36,2,2,3.0,0.0,-1,56.87058,25,0.000000,66.36,208.08,4810.8096,36.180
749996,749996,2,14,75.75,0,35.21,2,3,3.0,2.0,0,45.46242,21,0.026402,32.21,105.63,1239.7441,19.105
749997,749997,28,47,30.98,4,78.58,4,2,3.0,0.0,-1,15.26000,51,0.000000,75.58,235.74,6174.8164,40.790
749998,749998,41,42,108.98,4,45.39,4,2,3.0,0.0,-1,100.72939,47,0.000000,42.39,136.17,2060.2521,24.195


In [5]:
# Drop unnecessary columns
df.drop(columns=["id"],inplace=True)

In [6]:
# Drop unnecessary columns
X = df.drop(columns=["Listening_Time_minutes"])
y = df["Listening_Time_minutes"]

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
print("X_train shape:", X_train.shape)
print("X_valid shape:", X_valid.shape)
print("y_train shape:", y_train.shape)
print("y_valid shape:", y_valid.shape)

X_train shape: (600000, 16)
X_valid shape: (150000, 16)
y_train shape: (600000,)
y_valid shape: (150000,)


In [9]:
# ✅ STEP 6: Define Optuna objective function
def objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_lambda": trial.suggest_float("lambda", 1e-5, 1.0),
        "reg_alpha": trial.suggest_float("alpha", 1e-5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "objective": "reg:squarederror",
        "random_state": 42,
    }

    model = xgb.XGBRegressor(**params)

    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

    preds = model.predict(X_valid)
    rmse = sqrt(mean_squared_error(y_valid, preds))  # ✅ Fixed this line manually
    return rmse



In [10]:

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)  # You can increase this later if needed

[I 2025-04-06 14:44:26,849] A new study created in memory with name: no-name-2c9f98d3-8d78-4bda-accb-4e2bb638164c
[I 2025-04-06 14:45:51,123] Trial 0 finished with value: 12.838293457431773 and parameters: {'max_depth': 15, 'learning_rate': 0.03615814050272859, 'subsample': 0.5966268571603188, 'colsample_bytree': 0.9582055256564382, 'lambda': 0.5728927979497024, 'alpha': 0.5863222939191963, 'min_child_weight': 3, 'gamma': 2.755030100529278, 'n_estimators': 146}. Best is trial 0 with value: 12.838293457431773.
[I 2025-04-06 14:46:32,094] Trial 1 finished with value: 12.981867910285247 and parameters: {'max_depth': 7, 'learning_rate': 0.042554152352832164, 'subsample': 0.760183530600218, 'colsample_bytree': 0.8412006633506102, 'lambda': 0.0022995095996267245, 'alpha': 0.8003127978966041, 'min_child_weight': 1, 'gamma': 2.302447975785136, 'n_estimators': 594}. Best is trial 0 with value: 12.838293457431773.
[I 2025-04-06 14:47:06,871] Trial 2 finished with value: 13.248651892917906 and pa

In [11]:
print("Best Parameters:")
print(study.best_params)

print("\nBest Score (RMSE):")
print(study.best_value)

print("\nBest Trial:")
print(study.best_trial)

Best Parameters:
{'max_depth': 14, 'learning_rate': 0.023729272005270058, 'subsample': 0.8968090616429454, 'colsample_bytree': 0.8492576547683933, 'lambda': 0.9990102695713851, 'alpha': 0.38709502558228537, 'min_child_weight': 6, 'gamma': 3.823134356331538, 'n_estimators': 819}

Best Score (RMSE):
12.74118977664359

Best Trial:
FrozenTrial(number=34, state=1, values=[12.74118977664359], datetime_start=datetime.datetime(2025, 4, 6, 15, 44, 58, 923114), datetime_complete=datetime.datetime(2025, 4, 6, 15, 48, 29, 179142), params={'max_depth': 14, 'learning_rate': 0.023729272005270058, 'subsample': 0.8968090616429454, 'colsample_bytree': 0.8492576547683933, 'lambda': 0.9990102695713851, 'alpha': 0.38709502558228537, 'min_child_weight': 6, 'gamma': 3.823134356331538, 'n_estimators': 819}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'max_depth': IntDistribution(high=15, log=False, low=3, step=1), 'learning_rate': FloatDistribution(high=0.3, log=False, low=0.005, st

In [12]:
best_params={

    'max_depth': 14,
    'learning_rate': 0.023729272005270058,
    'subsample': 0.8968090616429454,
    'colsample_bytree': 0.8492576547683933,
    'lambda': 0.9990102695713851,
    'alpha': 0.38709502558228537,
    'min_child_weight': 6,
    'gamma': 3.823134356331538,
    'n_estimators': 819
}

In [13]:
code = xgb.XGBRegressor(**best_params)

In [14]:
code.fit(X_train,y_train)

XGBRegressor(alpha=0.38709502558228537, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8492576547683933, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=3.823134356331538, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.9990102695713851,
             learning_rate=0.023729272005270058, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=14, max_leaves=None,
             min_child_weight=6, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=819, ...)

In [15]:
code

XGBRegressor(alpha=0.38709502558228537, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8492576547683933, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=3.823134356331538, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.9990102695713851,
             learning_rate=0.023729272005270058, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=14, max_leaves=None,
             min_child_weight=6, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=819, ...)

In [16]:
y_out = code.predict(X_valid)

array([32.04776 , 33.617607, 39.89614 , ...,  1.621613, 75.899666,
       85.60913 ], dtype=float32)

In [17]:
test = pd.read_csv('/content/final_test.csv')

In [19]:
test.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Title_num,Ad_Density,Popularity_Diff,Popularity_Interaction,Host_Popularity_squared,Popularity_Average
0,750000,11,71,78.96,2,38.11,2,1,3.0,1.0,0,73,0.012664,35.11,114.33,1452.3721,20.555
1,750001,36,16,27.87,5,71.29,3,2,3.0,0.0,0,23,0.000000,68.29,213.87,5082.2641,37.145
2,750002,24,3,69.10,1,67.89,0,1,3.0,0.0,1,11,0.000000,64.89,203.67,4609.0521,35.445
3,750003,4,71,115.39,1,23.40,3,2,3.0,2.0,1,73,0.017332,20.40,70.20,547.5600,13.200
4,750004,27,46,72.32,4,58.10,6,2,3.0,2.0,0,50,0.027654,55.10,174.30,3375.6100,30.550


In [20]:
test.drop(columns=['id'],inplace =True)

In [21]:
y_xg = code.predict(test)

In [22]:
y_xg

array([54.416435, 18.793406, 47.94108 , ...,  7.747443, 71.83884 ,
       60.156498], dtype=float32)

In [23]:
sample = pd.read_csv('/content/sample_submission.csv')

In [24]:
sample['Listening_Time_minutes'] = y_xg

In [25]:
sample.to_csv('final_model_trained_using_xgboost.csv',index =False)